In [2]:
from PIL import Image
import numpy as np
import pandas as pd
from glob import glob
import os

In [3]:
def read_and_save_to_df(path):
    im = Image.open(path).convert('L')
    im = im.resize((50,50))
    im = np.array(im)/255
    data = im.flatten()
    return data

In [8]:
df_train = []
for img in glob('./bottle/train/good/*'):
    df_train.append(read_and_save_to_df(img))

In [6]:
df_test = []
real = []
for img in glob('./bottle/test/**/*'):
    df_test.append(read_and_save_to_df(img))
    real.append( 0 if 'good' in img else 1)

In [7]:
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, roc_curve, f1_score, classification_report

In [10]:
pca = PCA(n_components=100)
df_train_reduced = pca.fit_transform(df_train)
df_train_inverse = pca.inverse_transform(df_train_reduced)
train_loss = np.sum((df_train - df_train_inverse) ** 2, axis=1)
threshold = np.max(train_loss)*5
threshold

0.3557192103646907

# Test set

In [11]:
df_test_reduced = pca.transform(df_test)
df_test_inverse =pca.inverse_transform(df_test_reduced)

In [12]:
test_loss = np.sum((df_test - df_test_inverse) ** 2, axis=1)
predict = [1 if val>threshold else 0 for val in test_loss]

In [13]:
def print_metrics(real, predict):
    print('Accuracy: ', accuracy_score(real, predict))
    print('\nPrecision: ', precision_score(real, predict))
    print('\nrecall: ', recall_score(real, predict))
    print('\nf1_score: ', f1_score(real, predict))
    print('\nconfusion_matrix:\n ', pd.DataFrame(confusion_matrix(real, predict), index=[0, 1], columns=[0, 1]))
    print('\nclassification_report:\n ', classification_report(real, predict))

In [14]:
print_metrics(real, predict)

Accuracy:  0.963855421686747

Precision:  0.96875

recall:  0.9841269841269841

f1_score:  0.9763779527559054

confusion_matrix:
      0   1
0  18   2
1   1  62

classification_report:
                precision    recall  f1-score   support

           0       0.95      0.90      0.92        20
           1       0.97      0.98      0.98        63

    accuracy                           0.96        83
   macro avg       0.96      0.94      0.95        83
weighted avg       0.96      0.96      0.96        83



# most read book chapter for anomalies detection using pca

https://www.oreilly.com/library/view/hands-on-unsupervised-learning/9781492035633/ch04.html


PCA and AutoEncoder

https://towardsdatascience.com/machine-learning-for-anomaly-detection-and-condition-monitoring-d4614e7de770